In [ ]:
import yaml
from pprint import pprint
with open('../input/localnb001-transformers-ner/config.yml') as file:
    CFG = yaml.load(file, Loader=yaml.FullLoader)
pprint(CFG)

BATCH_SIZE = CFG["test_batch_size"]
TRAIN = False
MAX_LEN = CFG["max_len"]
USE_POS = CFG["use_pos"]
DEBUG = CFG["debug"]
TEXT_LEN = CFG["text_len"]
TAGS_VALS = CFG["tags_vals"]
USE_TPU = False

In [ ]:
if USE_TPU:
    ##### PyTorch-XLA installation via internet connection

    ##!cp -r ../input/localnb001-transformers-ner/* .
    #!cp -r ../input/d/riow1983/localnb001-transformers-ner/* .
    #!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev





    ##### PyTorch-XLA installation via source code (offline installation)

    # Credit to https://www.kaggle.com/joshi98kishan/foldtraining-pytorch-tpu-8-cores/data?scriptVersionId=48061653
    #Copying all the required wheel files from the created kaggle dataset to the working dir.
    !cp ../input/pytorch-xla-setup-script/torch-nightly-cp37-cp37m-linux_x86_64.whl ./torch-nightly-cp37-cp37m-linux_x86_64.whl
    !cp ../input/pytorch-xla-setup-script/torch_xla-nightly-cp37-cp37m-linux_x86_64.whl ./torch_xla-nightly-cp37-cp37m-linux_x86_64.whl
    !cp ../input/pytorch-xla-setup-script/torchvision-nightly-cp37-cp37m-linux_x86_64.whl ./torchvision-nightly-cp37-cp37m-linux_x86_64.whl

    # This deb files are the dependencies, copying them to the working dir.
    !cp ../input/pytorch-xla-setup-script/libgfortran4_7.5.0-3ubuntu1_18.04_amd64.deb ./libgfortran4_7.5.0-3ubuntu1_18.04_amd64.deb
    !cp ../input/pytorch-xla-setup-script/libomp5_5.0.1-1_amd64.deb ./libomp5_5.0.1-1_amd64.deb
    !cp ../input/pytorch-xla-setup-script/libopenblas-base_0.2.20ds-4_amd64.deb ./libopenblas-base_0.2.20ds-4_amd64.deb
    !cp ../input/pytorch-xla-setup-script/libopenblas-dev_0.2.20ds-4_amd64.deb ./libopenblas-dev_0.2.20ds-4_amd64.deb

    #installing pytorch-xla by running this script
    !python ../input/pytorch-xla-setup-script/pytorch-xla-env-setup.py --version nightly

    #Now, istalling depedencies
    !dpkg -i ./libgfortran4_7.5.0-3ubuntu1_18.04_amd64.deb
    !dpkg -i ./libomp5_5.0.1-1_amd64.deb
    !dpkg -i ./libopenblas-base_0.2.20ds-4_amd64.deb
    !dpkg -i ./libopenblas-dev_0.2.20ds-4_amd64.deb

    # Removing wheel and deb files, as we don't need them now.
    !rm torch-nightly-cp37-cp37m-linux_x86_64.whl 
    !rm torch_xla-nightly-cp37-cp37m-linux_x86_64.whl 
    !rm torchvision-nightly-cp37-cp37m-linux_x86_64.whl
    !rm libgfortran4_7.5.0-3ubuntu1_18.04_amd64.deb 
    !rm libomp5_5.0.1-1_amd64.deb 
    !rm libopenblas-base_0.2.20ds-4_amd64.deb 
    !rm libopenblas-dev_0.2.20ds-4_amd64.deb

In [ ]:
# Importing pytorch and the library for TPU execution

import torch
if USE_TPU:
    import torch_xla
    import torch_xla.core.xla_model as xm

In [ ]:
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel

if USE_TPU:
    # Preparing for TPU usage
    dev = xm.xla_device()
else:
    #dev = torch.device('cuda:0')
    dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(dev)

In [ ]:
# Defining some key variables that will be used later on in the training
# test = False
# use_pos = False
# CV = 1
# MAX_LEN = 290
# BATCH_SIZE = 4 #16
tokenizer = BertTokenizer.from_pretrained('../input/localnb001-transformers-ner/bert-base-uncased-vocab.txt')
#tokenizer = BertTokenizer.from_pretrained('bert-base-cased-vocab.txt')
#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
#tokenizer = BertTokenizer.from_pretrained(f'../input/localnb001-transformers-ner/bert-base-cased-ner-cv{CV}.pt')
#tokenizer = BertTokenizer.from_pretrained(f'../input/localnb001-transformers-ner')
#tokenizer = BertTokenizer.from_pretrained('../input/d/riow1983/localnb001-transformers-ner/bert-base-cased')

In [ ]:
!cp ../input/localnb001-transformers-ner/bridge.py ./bridge.py
#from bridge import *

import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import gc
import re
import pickle

import warnings
warnings.simplefilter('ignore')

if USE_POS:
    import spacy
    nlp = spacy.load("en_core_web_sm")

# Dataset

Credit to: https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/discussion/230341

In [ ]:
!python ./bridge.py {TRAIN} {MAX_LEN} {USE_POS} {DEBUG} {TEXT_LEN} {TAGS_VALS}

In [ ]:
sentences = pickle.load(open("./sentences.pkl", 'rb'))
print(sentences[0])

In [ ]:
num_labels = len(TAGS_VALS.split())

***
# Deprecated from here

In [ ]:
# def get_text(filename, test=test):
#     if test:
#         df = pd.read_json('../input/coleridgeinitiative-show-us-the-data/test/{}.json'.format(filename))
#     else:
#         df = pd.read_json('../input/coleridgeinitiative-show-us-the-data/train/{}.json'.format(filename))
#     text = " ".join(list(df['text']))
#     return text


# def clean_text(txt):
#     return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()




# def convert_tokens(x, m, max_len, train=False, verbose=False):
#     """
#     Args:
#         x: df row
#         m: row index
#     Returns:
#         df
    
#     ex) convert_tokens(row,i, MAX_LEN)
#     """
#     df = pd.DataFrame()
#     if use_pos:
#         text = x["tok"]
#         pos = x["pos"]
#     else:
#         text = x['text'].replace('\uf0b7','').split()

#     if train:
#         entity = x['dataset_label']

#         ## main
#         tokens=[]
#         k=0
#         for i,x in enumerate(text):

#             if k==0:
#                 if x==entity.split()[0]:
#                     entity_len = len(entity.split())
#                     if entity == ' '.join(text[i:i+entity_len]):
#                         tokens.extend(['o-dataset']*len(entity.split()))
#                         k = entity_len
#                     else:
#                         tokens.append('o')
#                 else:
#                     tokens.append('o')


#             k = max(0,k-1)
    

#     k=0
#     sentence_hash=[]
#     for i in range(0,len(text),max_len):
#         if verbose:
#             print("Is length of text[i:i+max_len] 290?", len(text[i:i+max_len]))
#         sentence_hash.extend([f'sentence#{k}']* len(text[i:i+max_len]))
#         k+=1

#     #df['word']=list(map(str,text))
#     df['word'] = text
#     if use_pos:
#         df['pos'] = pos
#     else:
#         df['pos'] = None
#     df['sentence'] = f'sentence{m}'
#     df['sentence#'] = sentence_hash
#     if train:
#         df['tag'] = tokens

#     return df

In [ ]:
# ## read data
# if test:
#     df_test = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")
# else:
#     df_test = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
# df_test['text'] = df_test['Id'].apply(get_text)

# ## clean text
# df_test["text"] = df_test["text"].apply(lambda x: clean_text(x))

# df_test

In [ ]:
# #num_labels = dataset["tag"].nunique() + 1
# num_labels = 3

In [ ]:
# # Creating a class to pull the words from the columns and create them into sentences

# class SentenceGetter(object):
    
#     def __init__(self, dataset):
#         self.n_sent = 1
#         self.dataset = dataset
#         self.empty = False
#         if use_pos:
#             agg_func = lambda s: [(w,p) for w,p in zip(s["word"].values.tolist(),
#                                                        s["pos"].values.tolist())]
#         else:
#             agg_func = lambda s: [(w,) for w in s["word"].values.tolist()]
#         self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
#         self.sentences = [s for s in self.grouped]
    
#     def get_next(self):
#         try:
#             s = self.grouped["Sentence: {}".format(self.n_sent)]
#             self.n_sent += 1
#             return s
#         except:
#             return None

# #### RIOW
# #getter = SentenceGetter(dataset)
# #### RIOWRIOW

# Deprecated above
***

- [x] ToDo: [localnb001-transformers-ner.ipynb](https://github.com/riow1983/Kaggle-Coleridge-Initiative/blob/main/notebooks/localnb001-transformers-ner.ipynb)を参考にDataloader, pred関数を実装する 
- [x] ToDo: finalテーブルをベースに, pred entityを加えたsubmission.csvを作成する処理を実装する

# Dataloader

In [ ]:
# example: https://huggingface.co/transformers/model_doc/bert.html
'''
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
'''

In [ ]:
#### RIOW
# class CustomDataset(Dataset):
#     #### RIOW
# #     def __init__(self, tokenizer, sentences, poses, max_len):
# #         self.len = len(sentences)
# #         self.sentences = sentences
# #         self.poses = poses
# #         self.tokenizer = tokenizer
# #         self.max_len = max_len
#     def __init__(self, tokenizer, df, max_len):
#         self.df = df
#         #self.lengths = [len(d) for d in self.df["text"]]
#         self.lengths = [np.ceil(len(d.split(" "))/MAX_LEN).astype(int) for d in self.df["text"]]
#         self.tokenizer = tokenizer
#         self.max_len = max_len
#     #### RIOWRIOW
    
#     def __len__(self):
#         #### RIOW
#         #return self.len
#         return sum(self.lengths)
#         #return sum([np.ceil(l/MAX_LEN).astype(int) for l in self.lengths])
#         #### RIOWRIOW
        
#     def index_converter(self, text_lengths, original_index, verbose=False):
#         # reference:
#         # [1] https://gist.github.com/Miladiouss/6ba0876f0e2b65d0178be7274f61ad2f
#         accum = np.add.accumulate(text_lengths)
#         sentence_index = len(np.argwhere(accum <= original_index))
#         if verbose:
#             print(f"*********** original_index: {original_index} *************")
#             print("sentence_index:", sentence_index)
#         #index_wrt_sentence = (original_index - np.insert(accum, 0, 0)[sentence_index]) // MAX_LEN
#         #index_wrt_sentence = np.floor((original_index - np.insert(accum, 0, 0)[sentence_index]) / MAX_LEN).astype(int)
#         index_wrt_sentence = original_index - np.insert(accum, 0, 0)[sentence_index]
#         if verbose:
#             print("index_wrt_sentence:", index_wrt_sentence)
#             print()
#         return sentence_index, index_wrt_sentence
        
#     def __getitem__(self, index):
#         #### RIOW
# #         sentence = str(self.sentences[index])
# #         if use_pos:
# #             pos = str(self.poses[index])
# #         else:
# #             pos = None
        
#         #df = self.df[index]
#         #sentence_index, index_wrt_sentence = self.index_converter(self.lengths, index*MAX_LEN)
#         sentence_index, index_wrt_sentence = self.index_converter(self.lengths, index, verbose=False)
#         #df = self.df[converted_index]
#         df = self.df[self.df.index==sentence_index]
#         # POS tagging
#         if use_pos:
#             tok, pos = [], []
#             #bar = tqdm(total = df.shape[0])
#             for doc in nlp.pipe(df['text'].values, batch_size=50, n_process=-1):
#                 if doc.is_parsed:
#                     tok.append([n.text for n in doc])
#                     pos.append([n.pos_ for n in doc])
#                 else:
#                     # We want to make sure that the lists of parsed results have the
#                     # same number of entries of the original Dataframe, so add some blanks in case the parse fails
#                     tok.append(None)
#                     pos.append(None)
#                 #bar.update(1)
#             df["tok"] = tok
#             df["pos"] = pos
#         else:
#             df["tok"] = None
#             df["pos"] = None
        
#         # process
#         dataset = pd.DataFrame()
#         #bar = tqdm(total = df.shape[0])
#         for i,row in df.iterrows():
#             _df = convert_tokens(row,i, MAX_LEN, verbose=False)
#             dataset = dataset.append(_df,ignore_index=True)
#             #bar.update(1)
            
#         dataset["sentence_idx"] = dataset["sentence"] + dataset["sentence#"]
#         #dataset = dataset[["sentence", "sentence_idx", "word", "pos"]].copy()
#         #dataset.rename(columns={"token":"word"}, inplace=True)
        
#         getter = SentenceGetter(dataset)
        
#         #print("getter.sentences:", getter.sentences)
#         #for sent in getter.sentences:
#         #    print("lenght of sent:", len(sent))
            
#         sentences = [' '.join([s[0] for s in sent]) for sent in getter.sentences]
#         if use_pos:
#             poses = [' '.join([s[1] for s in sent]) for sent in getter.sentences]
#         else:
#             poses = None
            
#         #print("len(sentences):", len(sentences))
#         #for sentence in sentences:
#         #    print("len(sentence.split(" ")):", len(sentence.split(" ")))
#         #print("sentences:", sentences)
#         #print("index_wrt_sentence:", index_wrt_sentence)
        
#         sentence = str(sentences[index_wrt_sentence])
#         if use_pos:
#             pos = str(poses[index_wrt_sentence])
#         else:
#             pos = None
#         #### RIOWRIOW
        
    
#         inputs = self.tokenizer.encode_plus(
#             sentence,
#             #### RIOW
#             #None,
#             pos,
#             #### RIOWRIOW
#             add_special_tokens=True,
#             max_length=self.max_len,
#             pad_to_max_length=True,
#             return_token_type_ids=True
#         )
#         ids = inputs['input_ids']
#         mask = inputs['attention_mask']

#         return {
#             'ids': torch.tensor(ids, dtype=torch.long),
#             'mask': torch.tensor(mask, dtype=torch.long)
#         } 
    
    
class CustomDataset(Dataset):
    def __init__(self, sentences, tokenizer, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        #self.poses = poses
        #self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        tmp = self.sentences[index]
        sentence = " ".join(tmp[0])
        
        if USE_POS:
            #pos = str(self.poses[index])
            pos = " ".join(tmp[1])
        else:
            #pos = self.poses # which is None
            pos = tmp[1]
        
        inputs = self.tokenizer.encode_plus(
            sentence,
            pos,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            pad_to_max_length=True, # future warning (deprecated)
            #padding=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        if TRAIN:
            #label = self.labels[index]
            label = list(tmp[2])
            label.extend([2]*self.max_len) # tag2idx = {'o':0, 'o-dataset':1, 'pad':2}
            label = label[:self.max_len]
        
            return {'ids': torch.tensor(ids, dtype=torch.long),
                    'mask': torch.tensor(mask, dtype=torch.long),
                    'tags': torch.tensor(label, dtype=torch.long)} 
        else:
            return {'ids': torch.tensor(ids, dtype=torch.long),
                    'mask': torch.tensor(mask, dtype=torch.long)} 
    
    def __len__(self):
        return self.len
    
    
    
#### RIOWRIOW

In [ ]:
#### RIOW
#testing_set = CustomDataset(tokenizer, sentences, poses, MAX_LEN)
#testing_set = CustomDataset(tokenizer, df_test, MAX_LEN)
testing_set = CustomDataset(sentences, tokenizer, MAX_LEN)
#### RIOWRIOW

# cf. 
'''Warning:
Truncation was not explicitly activated but `max_length` is provided a specific value, 
please use `truncation=True` to explicitly truncate examples to max length. 

... or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
'''

In [ ]:
test_params = {'batch_size': BATCH_SIZE,
               'shuffle': False,
               'num_workers': 8 #0
                }

testing_loader = DataLoader(testing_set, **test_params)

# Load the fine-tuned pre-trained BERT model

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        #self.l1 = transformers.BertForTokenClassification.from_pretrained('../input/localnb001-transformers-ner/bert-base-cased')
        #self.l1 = transformers.BertForTokenClassification.from_pretrained(f'../input/localnb001-transformers-ner/bert-base-cased-ner-cv{CV}.pth', num_labels=num_labels)
        #self.l1 = transformers.BertForTokenClassification.from_pretrained(f'../input/localnb001-transformers-ner/bert-base-cased-ner-cv{CV}.bin')
        #self.l1 = transformers.BertForTokenClassification.from_pretrained('../input/d/riow1983/localnb001-transformers-ner', num_labels=num_labels)
        self.l1 = transformers.BertForTokenClassification.from_pretrained('../input/localnb001-transformers-ner', num_labels=num_labels)
        #self.l1 = transformers.BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=num_labels) # This requires internet connection.
        
        # self.l2 = torch.nn.Dropout(0.3)
        # self.l3 = torch.nn.Linear(768, 200)
    
    def forward(self, ids, mask, labels):
        output_1= self.l1(ids, mask, labels = labels)
        # output_2 = self.l2(output_1[0])
        # output = self.l3(output_2)
        return output_1

In [ ]:
model = BERTClass()

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
print(USE_TPU)
print(dev)

Save on CPU, Load on GPU (Example)
```
# Save
torch.save(net.state_dict(), PATH)

# Load
device = torch.device("cuda")
model = Net()
# Choose whatever GPU device number you want
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))
# Make sure to call input = input.to(device) on any input tensors that you feed to the model
model.to(device)
```
refenrece: https://pytorch.org/tutorials/recipes/recipes/save_load_across_devices.html

In [ ]:
# ToDo: cased -> uncased
#output_model = f"../input/localnb001-transformers-ner/bert-base-cased-ner-pad-cv{CV}.pth"
CV = 1
if USE_POS:
    output_model = f"../input/localnb001-transformers-ner/bert-base-cased-ner-pad-cv{CV}-epochs5.pth"
else:
    output_model = f"../input/localnb001-transformers-ner/bert-base-cased-ner-pad-nopos-cv{CV}-epochs5.pth"

if USE_TPU:
    checkpoint = torch.load(output_model, map_location='tpu')
else:
    #checkpoint = torch.load(output_model, map_location=torch.device('cpu'))
    checkpoint = torch.load(output_model, map_location="cuda:0")

model.load_state_dict(checkpoint['model_state_dict'])
model.to(dev)

~~ToDo: fine-tuned modelの保存ファイルの拡張子は.pthではなく.binにする~~  
~~reference: https://github.com/huggingface/transformers/issues/1620~~

# Prediction

In [ ]:
#tags_vals = ['o-dataset', 'o']
#tags_vals = ['o', 'o-dataset', 'pad'] # the order is the same as localnb001
tags_vals = TAGS_VALS.split()

In [ ]:
def pred(model, testing_loader, verbose=False):
    model.eval()
    
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(dev, dtype = torch.long)
            print(_, ids.shape)
            mask = data['mask'].to(dev, dtype = torch.long)

            #output = model(ids, mask, mask)
            output = model(ids, mask, labels=None)
            #loss, logits = output[:2]
            logits = output[0]
            logits = logits.detach().cpu().numpy()
            
            predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
            if verbose:
                #print("predictions:", predictions)
                print("length of predictions:", len(predictions))
                for p in predictions:
                    print("length of p:", len(p))
                print()
                
        pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
        if verbose:
            print("pred_tags:", pred_tags)
            print()
        
    return pred_tags

In [ ]:
# To get the results on the validation set. This data is not seen by the model

pred_tags = pred(model, testing_loader, verbose=False)

'''Warning:
Truncation was not explicitly activated but `max_length` is provided a specific value, 
please use `truncation=True` to explicitly truncate examples to max length. 
Defaulting to 'longest_first' truncation strategy. 

If you encode pairs of sequences (GLUE-style) with the tokenizer 
you can select this strategy more precisely by providing a specific strategy to `truncation`.


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: 
The `pad_to_max_length` argument is deprecated and will be removed in a future version, 
use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, 
or use `padding='max_length'` to pad to a max length. 
In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or 
leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
'''

In [ ]:
# pred_tags 内容確認
np.unique(pred_tags)
# EPOCHS=1では全て'o'で予測してしまっていた
# EPOCHS=5では逆に全て'o-dataset'で予測してしまっている

In [ ]:
#print(len(pred_tags), pred_tags[:10])
# 175 * MAX_LEN(=290) = 50750

finalテーブルをベースに, pred entityを加えたsubmission.csvを作成する処理を実装する

In [ ]:
# Id = "2100032a-7c33-4bff-97ef-690822c43466"
# pstns = [tmp[-1]==Id for tmp in sentences]
# positions_contracted = np.array(pstns)
# positions_contracted

In [ ]:
# Id = "2100032a-7c33-4bff-97ef-690822c43466"
# pstns = [tmp[-1]==Id for tmp in sentences]
# positions_contracted = np.array(pstns)
# positions_dilated = np.array([p for p in positions for i in range(MAX_LEN)])
# print(np.sort(np.where(positions_dilated==1)[0]))

In [ ]:
#np.array(sentences)[positions_contracted]

In [ ]:
# tmp = pd.read_json(f'../input/coleridgeinitiative-show-us-the-data/test/{Id}.json')
# text = " ".join(list(tmp['text']))
# text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
# text[:100] #string

In [ ]:
# text = np.array(text.split())
# text

In [ ]:
# np.ceil(len(text)/MAX_LEN)*MAX_LEN

In [ ]:
#text = np.pad(text, (0, int(np.ceil(len(text)/MAX_LEN)*MAX_LEN)-len(text)), 'constant', constant_values="[PAD]")

In [ ]:
#"|".join(text[np.array([0,1,2])]).strip("|")

In [ ]:
# Id = "2f392438-e215-4169-bebf-21ac4ff253e1"
# positions = [tmp[-1]==Id for tmp in sentences]
# positions = np.array([p for p in positions for i in range(MAX_LEN)])
# print(np.sort(np.where(positions==1)[0]))

In [ ]:
# Id = "3f316b38-1a24-45a9-8d8c-4e05a42257c6"
# positions = [tmp[-1]==Id for tmp in sentences]
# positions = np.array([p for p in positions for i in range(MAX_LEN)])
# print(np.sort(np.where(positions==1)[0]))

In [ ]:
# Id = "8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60"
# positions = [tmp[-1]==Id for tmp in sentences]
# positions = np.array([p for p in positions for i in range(MAX_LEN)])
# print(np.sort(np.where(positions==1)[0]))

In [ ]:
#np.array(pred_tags)[positions]

In [ ]:
#np.array(pred_tags)[np.array([0,1,2])]

In [ ]:
def make_submission_file(df, sentences, pred_tags, finding_tag="o-dataset", verbose=False):
    """
    Args:
        df: pd.DataFrame
        sentences: List[tuple[np.array, ..., str]]
        pred_tagas: List[str]
    Returns:
        df: pd.DataFrame
    """
    #df["PredictionString"] = ""
    for i,row in tqdm(df.iterrows()):
        Id = row["Id"]
        pstns = [tmp[-1]==Id for tmp in sentences]
        #positions_contracted = np.array(pstns)
        positions_dilated = np.array([p for p in pstns for i in range(MAX_LEN)])
        
        pred_tags_per_pub = np.array(pred_tags)[positions_dilated]
        
        tmp = pd.read_json(f'../input/coleridgeinitiative-show-us-the-data/test/{Id}.json')
        text = " ".join(list(tmp['text']))
        text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip() #clean_text
        
        text = np.array(text.split()) # to numpy array
        
        if TEXT_LEN > 0:
            text = text[:TEXT_LEN]
        
        desired_len = int(np.ceil(len(text)/MAX_LEN)*MAX_LEN) # padding
        text = np.pad(text, (0, desired_len-len(text)), 'constant', constant_values="[PAD]")
        
        if verbose:
            print("len(pred_tags_per_pub): ", len(pred_tags_per_pub))
            print("len(text): ", len(text))
        
        assert len(pred_tags_per_pub) == len(text)
        cond = np.where(pred_tags_per_pub==finding_tag)[0] # extract finding tokens
        pred_string = "|".join(text[cond]).strip("|")
        #pred_string = " ".join(np.where(pred_tags_per_pub==finding_tag, text, "|")).strip("|")
        
        if verbose:
            print("row#: ", i)
            print("pred_string:\n", pred_string, type(pred_string))
        
        df.loc[i, "PredictionString"] = pred_string
        #df.at[i, "PredictionString"] = pred_string
    
    return df

In [ ]:
df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")
df = make_submission_file(df, sentences, pred_tags, finding_tag="o-dataset", verbose=False)
df

In [ ]:
df.to_csv("submission.csv", index=False)

In [ ]:
# Id = "8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60"
# tmp = pd.read_json(f'../input/coleridgeinitiative-show-us-the-data/test/{Id}.json')
# text = " ".join(list(tmp['text']))
# text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
# text[:100] #string

In [ ]:
# def extractor(df, pred_tags, max_len):
#     """
#     Args:
#         df: pd.DataFrame
#         pred_tags: List of Str
#         max_len: Int
#     Returns:
#         df: pd.DataFrame
#     """
#     df["PredictionString"] = ""
#     total_length = 0
#     for i, row in df.iterrows():
#         wordlist = row["text"].split(" ")
#         units = len(wordlist) // max_len
#         resid = len(wordlist) - (units*max_len)
        
#         first = wordlist[:units*max_len]
#         if resid > 0:
#             second = wordlist[-resid:]
#             second = np.pad(second, (0, max_len-len(second)), 'constant', constant_values="[PAD]")
#             first = np.append(first, second)
#         first = np.array(first)
#         length = len(first)
#         sub_pred_tags = np.array(pred_tags[total_length:total_length+length])
        
#         datalist = first[np.where(sub_pred_tags=='o-dataset')[0]]
#         datastring = "|".join(datalist).strip("|")
#         df.at[i, "PredictionString"] = datastring
#         total_length += length
#     return df

In [ ]:
# df = extractor(df_test, pred_tags, MAX_LEN)
# df

In [ ]:
# del df_test, pred_tags
# gc.collect()

In [ ]:
#df.to_csv("submission.csv", index=False)